In [1]:
import dask.distributed
from dask.distributed import Client, as_completed, get_worker
import numpy as np
import time
import os
from pyretis.setup import create_simulation
from pyretis.core.tis import shoot, wire_fencing
from pyretis.core.retis import retis_swap_zero # need to disable "add_path_data()"
from pyretis.inout.settings import parse_settings_file
from infretis import REPEX_state, calc_cv_vector
dask.config.config['work-stealing'] = False

In [2]:
# Connect to scheduler
n_workers = 2
client = Client(n_workers=n_workers, )

2022-11-01 08:56:54,550 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-pxoeuzin', purging
2022-11-01 08:56:54,550 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-ahhn3050', purging


In [3]:
# %%capture
# Set up simulation variables
INP = './retis.rst'
sim_settings = parse_settings_file(INP)
sim = create_simulation(sim_settings)
sim.set_up_output(sim_settings, progress=True)
sim.initiate(sim_settings)


CREATING  ENSEMBLE  0
Initializing unit system.
Creating system from settings.
Creating force field.
Created units: "gromacs".
Created engine "GROMACS engine" from settings.

CREATING  ENSEMBLE  1
Initializing unit system.
Creating system from settings.
Creating force field.
Created units: "gromacs".
Created engine "GROMACS engine" from settings.

CREATING  ENSEMBLE  2
Initializing unit system.
Creating system from settings.
Creating force field.
Created units: "gromacs".
Created engine "GROMACS engine" from settings.

CREATING  ENSEMBLE  3
Initializing unit system.
Creating system from settings.
Creating force field.
Created units: "gromacs".
Created engine "GROMACS engine" from settings.

CREATING  ENSEMBLE  4
Initializing unit system.
Creating system from settings.
Creating force field.
Created units: "gromacs".


CRITICAL:pyretis.inout.fileio:Could not open file "load/000/energy.txt" for reading
CRITICAL:pyretis.inout.fileio:I/O error ({2}): {No such file or directory}
CRITICAL:pyretis.initiation.initiate_load:Initial path for [0^-] ends at the wrong interface!


Created engine "GROMACS engine" from settings.
Will initiate paths using method "load".

Loading data for path path ensemble [0^-]
Copying trajectory files.
Loading order parameters.
Creating path from files.
Loading energy data.
Could not read file: load/000/energy.txt
Energies are not set.
Sorting input files
Found initial path for [0^-]:
- Path with length 179 (max: None)
- Order parameter max: (0.270052, 0)
- Order parameter min: (0.263395, 89)
- Start 0.270052
- End 0.270052
- Status: The path has been accepted
- Generated: Path was loaded from unformatted external file(s)
- Weight: 1.0



CRITICAL:pyretis.inout.fileio:Could not open file "load/001/energy.txt" for reading
CRITICAL:pyretis.inout.fileio:I/O error ({2}): {No such file or directory}
CRITICAL:pyretis.initiation.initiate_load:Initial path for [0^+] starts at the wrong interface!


Loading data for path path ensemble [0^+]
Copying trajectory files.
Loading order parameters.
Creating path from files.
Loading energy data.
Could not read file: load/001/energy.txt
Energies are not set.
Sorting input files
Found initial path for [0^+]:
- Path with length 109 (max: None)
- Order parameter max: (0.279931, 54)
- Order parameter min: (0.26981, 0)
- Start 0.26981
- End 0.26981
- Status: The path has been accepted
- Generated: Path was loaded from unformatted external file(s)
- Weight: 1.0



...

CRITICAL:pyretis.inout.fileio:Could not open file "load/003/energy.txt" for reading
CRITICAL:pyretis.inout.fileio:I/O error ({2}): {No such file or directory}
CRITICAL:pyretis.initiation.initiate_load:Initial path for [2^+] ends at the wrong interface!


Loading data for path path ensemble [2^+]
Copying trajectory files.
Loading order parameters.
Creating path from files.
Loading energy data.
Could not read file: load/003/energy.txt
Energies are not set.
Sorting input files
Found initial path for [2^+]:
- Path with length 61 (max: None)
- Order parameter max: (0.307148, 30)
- Order parameter min: (0.269896, 0)
- Start 0.269896
- End 0.269896
- Status: The path has been accepted
- Generated: Path was loaded from unformatted external file(s)
- Weight: 57.0



CRITICAL:pyretis.inout.fileio:Could not open file "load/004/energy.txt" for reading
CRITICAL:pyretis.inout.fileio:I/O error ({2}): {No such file or directory}
CRITICAL:pyretis.initiation.initiate_load:Initial path for [3^+] ends at the wrong interface!


Loading data for path path ensemble [3^+]
Copying trajectory files.
Loading order parameters.
Creating path from files.
Loading energy data.
Could not read file: load/004/energy.txt
Energies are not set.
Sorting input files
Found initial path for [3^+]:
- Path with length 101 (max: None)
- Order parameter max: (0.336438, 50)
- Order parameter min: (0.269994, 0)
- Start 0.269994
- End 0.269994
- Status: The path has been accepted
- Generated: Path was loaded from unformatted external file(s)
- Weight: 97.0



True

In [4]:
def run_md(ens_num, input_traj, settings, ensembles, cycle, move, pin):
    interfaces = settings['simulation']['interfaces']
    worker = get_worker()
    pin0 = str(worker.name)
    start_time = time.time() 
    out = {'ensembles': [],                                                  
           'accepted_trajs': [],                                        
           'traj_vectors': [],
           'status': None,
           'pin': pin,
           'move': move,
           'pin0': [pin, pin0]}
    path_numbers_old = []
    for traj0 in input_traj:
        path_numbers_old.append(traj0.path_number)
    out['path_numbers_old'] = path_numbers_old
    
    if len(ens_num) == 1:
        start_cond = ensembles[ens_num[0]+1]['path_ensemble'].start_condition
        tis_settings = settings['ensemble'][ens_num[0]+1]['tis'] 
        
        if move == 'sh':
            accept, trial, status = shoot(ensembles[ens_num[0]+1],
                                          tis_settings,
                                          start_cond)
        else:
            accept, trial, status = wire_fencing(ensembles[ens_num[0]+1],
                                                 tis_settings,
                                                 start_cond)
        
        if accept:                                                                  
            out_traj = trial                                                        
        else:                                                                       
            out_traj = input_traj[0]
        if ens_num[0] < 0:
            interfaces = interfaces[0:1]
        cv_vector = calc_cv_vector(out_traj, interfaces, move)
                
        out['ensembles'] = list(ens_num)
        out['accepted_trajs'] = [out_traj]
        out['traj_vectors'] = [cv_vector]
        out['status'] = status
    else:
        accept, trial, status = retis_swap_zero(ensembles, 
                                                settings,
                                                cycle)
        if accept:                                                                  
            out_traj = trial                                                        
        else:                                                                       
            out_traj = input_traj

        out['ensembles'] = list(ens_num)
        out['accepted_trajs'] = out_traj
        ifaces = [[interfaces[0:1]], interfaces]
        for traj, iface in zip(out_traj, ifaces):
            out['traj_vectors'].append(calc_cv_vector(traj, iface, move))     
        out['status'] = status
    curr_time = time.time() 
    out['time'] = curr_time - start_time
    return out


def print_path_info(state, intfs=False):
    ens_no = len(state._trajs[:-1])
    ens_str = [f'{i:03.0f}' for i in range(ens_no)]
    state_dic = {}
    path_dic = {}
    ens_pwds = []
    locks = [traj0.path_number for traj0, lock0 in
             zip(state._trajs[:-1], state._locks[:-1]) if lock0]

    
    for path_temp in state._trajs[:-1]:
        path_pwds = sorted(set([pp.particles.config[0] for pp in path_temp.phasepoints]))
        ens = path_pwds[0].split('/')[4]
        state_dic[ens] = {'pwds': [pwd.split('/')[-1] for pwd in path_pwds]}
        state_dic[ens]['path_number'] = path_temp.path_number
        path_dic[state_dic[ens]['path_number']] = [pwd.split('k')[-1] for pwd in path_pwds]

    for ens in ens_str:
        ens_pwds.append(sorted(os.listdir(f'./{ens}/accepted')))

    # check if state_paths correspond to path_pwds:
    for ens, string1 in zip(ens_str, ens_pwds):
        string0 = state_dic[ens]['pwds']
        if string0 != string1:
            print(string0, string1)
            print('warning! the state_paths does not correspond to the path_pwds!')
    
    last_prob = True
    if type(state._last_prob) == type(None):
        state.prob
        last_prob = False
        
    # print current state
    p_num = [state_dic[ens]['path_number'] for ens in ens_str]
    print('===')
    print(' xx |\t', '\t'.join(['e'+i for i in ens_str]))
    print(' -- |     -----------------------------------')
    
    live_trajs = [traj.path_number for traj in state._trajs[:-1]] # state.live_paths()
    w_start = 0
    for live in live_trajs:
        if live not in locks:
            for weight in state._last_prob[w_start:-1]:
                w_start += 1
                if sum(weight) != 0:
                    print(f'p{live:02.0f} |\t',
                          '\t'.join([f'{j:.2f}' if j != 0 else '----' for j in weight[:-1]]))
                    traj_num_dic[live]['weight'] += weight
                    break
        else:
            print(f'p{live:02.0f} |\t', '\t'.join(['----' for j in range(ens_no)]))

    print('===')
    if intfs:
        print(len(state._trajs[:-1]))
        for live in state._trajs[:-1]:
            print(live.path_number, calc_cv_vector(live, interfaces, 'sh'))
    print('===')
            
    if not last_prob:
        state._last_prob = None
        
def set_shooting(sim, ens, input_traj, pin, move):
    print('shooting', move, 'in ensembles:', ' '.join([f'00{ens_num+1}' for ens_num in ens]),
          'with paths:', ' '.join([str(trajj.path_number) for trajj in input_traj]),
          'and worker:', pin)

    # assign gromacs resurces to workers:
    for ens_num, traj_inp in zip(ens, input_traj):
        ens_num += 1
        not_pin = '1' if pin == '0' else '0'
        mdrun = sim.ensembles[ens_num]['engine'].mdrun.replace(not_pin, pin)
        mdrun_c = sim.ensembles[ens_num]['engine'].mdrun_c.replace(not_pin, pin)
        sim.ensembles[ens_num]['engine'].mdrun = mdrun
        sim.ensembles[ens_num]['engine'].mdrun_c = mdrun_c
        sim.ensembles[ens_num]['path_ensemble'].last_path = traj_inp.copy()

def treat_output(output, state, sim, traj_num_dic, traj_num, save=False):
    ensembles = output['ensembles']
    out_trajs = output['accepted_trajs']
    traj_vectors = output['traj_vectors']
    status = output['status']
    path_numbers_old = output['path_numbers_old']
    pin = output['pin']
    move = output['move']
    pin0 = output['pin0']
    time_spent = output['time']
    
    # analyse and record worker data
    for ens_num, out_traj, traj_v, pn_old in zip(ensembles,
                                                 out_trajs,
                                                 traj_vectors,
                                                 path_numbers_old):
        # if path is new: number and save the path:
        if out_traj.path_number == None or status == 'ACC':
            out_traj.path_number = traj_num
            ens_save_idx = traj_num_dic[pn_old]['ens_idx']
            traj_num_dic[traj_num] = {'weight': np.zeros(size+1),
                                      'adress': set(kk.particles.config[0].split('salt')[-1] 
                                                    for kk in out_traj.phasepoints),
                                      'ens_idx': ens_save_idx}
            traj_num += 1
            sim.ensembles[ens_save_idx]['path_ensemble'].store_path(out_traj)
            
            cycle = {'step': traj_num -1 , 'endcycle': 10, 'startcycle': 0, 'stepno': 10, 'steps': 10}
            result = {f'status-{ens_num+1}': 'ACC', 'cycle': cycle, f'path-{ens_num+1}':  out_traj,
                      f'accept-{ens_num+1}': True, f'move-{ens_num+1}': 'sh', 
                      'all-2': {'ensemble_number': ens_num+1, 'mc-move': 'sh',
                                'status': 'ACC', 'trial': out_traj, 'accept': True},
                      f'pathensemble-{ens_num+1}': sim.ensembles[0]['path_ensemble']}
            
            # NB! Saving can take some time..
            if save:
                for task in sim.output_tasks:
                    task.output(result)
            
        state.add_traj(ens_num, out_traj, traj_v)
        
        # record weights 
        live_trajs = [traj.path_number for traj in state._trajs[:-1]] # state.live_paths()
        
        traj_lock = [traj0.path_number for traj0, lock0 in
                     zip(state._trajs[:-1], state._locks[:-1]) if lock0]
        w_start = 0
        for live in live_trajs:
            if live not in traj_lock:
                for weight in state._last_prob[w_start:-1]:
                    w_start += 1
                    if sum(weight) != 0:
                        traj_num_dic[live]['weight'] += weight
                        break
    
    # print information to screen
    print('shooted', move, 'in ensembles:', ' '.join([f'00{ens_num+1}' for ens_num in ensembles]),
          'with paths:', ' '.join([str(pn_old) for pn_old in path_numbers_old]), '->', 
          ' '.join([str(trajj.path_number) for trajj in out_trajs]),
          'with status:', status, 'and worker:', pin, f'total time: {time_spent:.2f}')
    print('cobra0', pin0)
    return traj_num_dic, traj_num, pin

In [5]:
# Define some variables
interfaces = sim.settings['simulation']['interfaces']
ens_str = [f'{i:03.0f}' for i in range(len(sim.settings['simulation']['interfaces']))]
size=len(interfaces)
traj_num_dic = {}            # store traj data 
traj_num = 0                 # numbering of accepted paths
moves = sim.settings['tis']['shooting_moves']
move_d = {i: move for i, move in zip(list(range(len(interfaces))), moves)}

stopping = 100

# Setup the infretis simulation
state = REPEX_state(size, minus=True)


for i in range(size-1):
    # we add all the i+ paths.
    path = sim.ensembles[i+1]['path_ensemble'].last_path
    path.path_number = traj_num
    state.add_traj(ens=i, traj=path,
                   valid=calc_cv_vector(path, interfaces, move_d[i+1]),
                   count=False)
    traj_num_dic[traj_num] = {'weight': np.zeros(size+1), 
                              'adress':  set(kk.particles.config[0].split('salt')[-1]
                                             for kk in path.phasepoints),
                              'ens_idx': traj_num + 1}
    traj_num += 1

# add minus path:
path = sim.ensembles[0]['path_ensemble'].last_path
path.path_number = traj_num
state.add_traj(ens=-1, traj=path, valid=(1,), count=False)
traj_num_dic[traj_num] = {'weight': np.zeros(size+1),
                          'adress':  set(kk.particles.config[0].split('salt')[-1]
                                         for kk in path.phasepoints),
                          'ens_idx': 0}
traj_num += 1

# Define an iterator that is going to be our results queue
futures = as_completed(None, with_results=True)

# Start running
loop = 0

In [6]:
print('stored ensemble paths:')
print(' '.join([f'00{i}: {j},' for i, j in enumerate([sim0['path_ensemble'].last_path.path_number for sim0 in sim.ensembles])]))
print('saved ensemble paths:', [last_path0.path_number for last_path0 in state._trajs[:-1]])
print_path_info(state)
print(' ')

# initialization: submit the first batch of workers
for worker in range(n_workers):
    
    print(f'------- infinity {loop} START -------')
    
    # Pick ens and trajs
    ens, input_traj = state.pick()
    if len(ens) > 1 or ens[0] == -1:
        move = 'sh'
    else:
        move = move_d[ens[0]+1]
    
    # Print & assign worker to pin
    set_shooting(sim, ens, input_traj, str(worker), move)
    
    # submit job
    fut = client.submit(run_md, ens, input_traj, sim.settings, sim.ensembles, loop, move, str(worker), pure=False)
    futures.add(fut)
    
    print(f'------- infinity {loop} END -------')
    loop += 1
    print()

# while loop
while loop < stopping:
    
    # get output from finished worker
    output = next(futures)[1]
    print(f'------- infinity {loop} START -------')

    
    # analyze & store output
    start_time = time.time()
    traj_num_dic, traj_num, pin = treat_output(output, state, sim,
                                               traj_num_dic, traj_num)
    print('time spent swapping and saving paths:', f'{time.time() - start_time:.02f}')
    
    # print analyzed output
    print_path_info(state)
    
    # chose ens and path for the next job
    ens, input_traj = state.pick()
    if len(ens) > 1:
        move = 'sh'
    else:
        move = move_d[ens[0]+1]
        
    # print & assign worker to pin 
    set_shooting(sim, ens, input_traj, pin, move)
    
    # submit job
    fut = client.submit(run_md, ens, input_traj, sim.settings, sim.ensembles, loop, move, pin, pure=False)
    futures.add(fut)

    print(f'------- infinity {loop} END -------')
    loop += 1

    print()

# get the last futures out
for output_l in futures:
    output = output_l[1]
    print(f'------- infinity {loop} START -------')

    traj_num_dic, traj_num, pin = treat_output(output, state, sim,
                                               traj_num_dic, traj_num)
    
    print_path_info(state)
    print(f'------- infinity {loop} END -------')
    loop += 1
    print()

stored ensemble paths:
000: 4, 001: 0, 002: 1, 003: 2, 004: 3,
saved ensemble paths: [4, 0, 1, 2, 3]
DEBUG this should not happen outside of wirefencing
===
 xx |	 e000	e001	e002	e003	e004
 -- |     -----------------------------------
p04 |	 1.00	----	----	----	----
p00 |	 ----	1.00	----	----	----
p01 |	 ----	----	0.68	0.32	----
p02 |	 ----	----	0.32	0.68	----
p03 |	 ----	----	----	----	1.00
===
===
 
------- infinity 0 START -------
DEBUG this should not happen outside of wirefencing
shooting wf in ensembles: 002 with paths: 1 and worker: 0
------- infinity 0 END -------

------- infinity 1 START -------
DEBUG this should not happen outside of wirefencing
shooting wf in ensembles: 003 with paths: 2 and worker: 1
------- infinity 1 END -------

------- infinity 2 START -------
DEBUG this should not happen outside of wirefencing
shooted wf in ensembles: 003 with paths: 2 -> 5 with status: ACC and worker: 1 total time: 17.98
cobra0 ['1', '1']
time spent swapping and saving paths: 0.01
==

/home/daniel/dask_venv/lib/python3.8/site-packages/distributed/worker.py:2938: UserWarning: Large object of size 1.03 MiB detected in task graph: 
  ((0,), (<pyretis.core.path.Path object at 0x7f53c4 ... , 3, 'sh', '0')
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  warnings.warn(


------- infinity 4 START -------
DEBUG this should not happen outside of wirefencing
shooted sh in ensembles: 000 with paths: 4 -> 7 with status: ACC and worker: 1 total time: 6.13
cobra0 ['1', '1']
time spent swapping and saving paths: 0.01
===
 xx |	 e000	e001	e002	e003	e004
 -- |     -----------------------------------
p07 |	 1.00	----	----	----	----
p00 |	 ----	----	----	----	----
p06 |	 ----	----	0.76	0.24	----
p05 |	 ----	----	0.24	0.76	----
p03 |	 ----	----	----	----	1.00
===
===
shooting wf in ensembles: 002 with paths: 5 and worker: 1
------- infinity 4 END -------

------- infinity 5 START -------
DEBUG this should not happen outside of wirefencing
shooted sh in ensembles: 001 with paths: 0 -> 8 with status: ACC and worker: 0 total time: 7.57
cobra0 ['0', '0']
time spent swapping and saving paths: 0.01
===
 xx |	 e000	e001	e002	e003	e004
 -- |     -----------------------------------
p07 |	 1.00	----	----	----	----
p08 |	 ----	1.00	----	----	----
p05 |	 ----	----	----	----	---

------- infinity 18 START -------
DEBUG this should not happen outside of wirefencing
shooted wf in ensembles: 002 with paths: 22 -> 25 with status: ACC and worker: 1 total time: 18.23
cobra0 ['1', '1']
time spent swapping and saving paths: 0.01
===
 xx |	 e000	e001	e002	e003	e004
 -- |     -----------------------------------
p24 |	 1.00	----	----	----	----
p23 |	 ----	0.47	0.39	0.13	----
p25 |	 ----	0.39	0.44	0.17	----
p18 |	 ----	0.14	0.16	0.70	----
p14 |	 ----	----	----	----	----
===
===
shooting wf in ensembles: 003 with paths: 18 and worker: 1
------- infinity 18 END -------

------- infinity 19 START -------
DEBUG this should not happen outside of wirefencing
shooted wf in ensembles: 004 with paths: 14 -> 26 with status: ACC and worker: 0 total time: 22.34
cobra0 ['0', '0']
time spent swapping and saving paths: 0.02
===
 xx |	 e000	e001	e002	e003	e004
 -- |     -----------------------------------
p24 |	 1.00	----	----	----	----
p23 |	 ----	0.54	0.46	----	----
p25 |	 ----	0.46	0.5

------- infinity 32 START -------
DEBUG this should not happen outside of wirefencing
shooted sh in ensembles: 000 with paths: 38 -> 40 with status: ACC and worker: 0 total time: 5.57
cobra0 ['0', '0']
time spent swapping and saving paths: 0.01
===
 xx |	 e000	e001	e002	e003	e004
 -- |     -----------------------------------
p40 |	 1.00	----	----	----	----
p39 |	 ----	1.00	----	----	----
p37 |	 ----	----	----	----	----
p36 |	 ----	----	----	0.43	0.57
p33 |	 ----	----	----	0.57	0.43
===
===
shooting sh in ensembles: 000 with paths: 40 and worker: 0
------- infinity 32 END -------

------- infinity 33 START -------
DEBUG this should not happen outside of wirefencing
shooted sh in ensembles: 000 with paths: 40 -> 41 with status: ACC and worker: 0 total time: 4.78
cobra0 ['0', '0']
time spent swapping and saving paths: 0.01
===
 xx |	 e000	e001	e002	e003	e004
 -- |     -----------------------------------
p41 |	 1.00	----	----	----	----
p39 |	 ----	1.00	----	----	----
p37 |	 ----	----	----	

------- infinity 46 START -------
DEBUG this should not happen outside of wirefencing
shooted wf in ensembles: 002 with paths: 55 -> 57 with status: ACC and worker: 1 total time: 19.64
cobra0 ['1', '1']
time spent swapping and saving paths: 0.03
===
 xx |	 e000	e001	e002	e003	e004
 -- |     -----------------------------------
p53 |	 ----	----	----	----	----
p54 |	 ----	----	----	----	----
p57 |	 ----	----	1.00	----	----
p56 |	 ----	----	----	0.33	0.67
p52 |	 ----	----	----	0.67	0.33
===
===
shooting wf in ensembles: 004 with paths: 56 and worker: 1
------- infinity 46 END -------

------- infinity 47 START -------
DEBUG this should not happen outside of wirefencing
DEBUG this should not happen outside of wirefencing
shooted sh in ensembles: 000 001 with paths: 53 54 -> 58 59 with status: ACC and worker: 0 total time: 6.09
cobra0 ['0', '0']
time spent swapping and saving paths: 0.02
===
 xx |	 e000	e001	e002	e003	e004
 -- |     -----------------------------------
p58 |	 1.00	----	----	-

------- infinity 59 END -------

------- infinity 60 START -------
DEBUG this should not happen outside of wirefencing
DEBUG this should not happen outside of wirefencing
shooted sh in ensembles: 000 001 with paths: 71 72 -> 74 75 with status: ACC and worker: 0 total time: 8.33
cobra0 ['0', '0']
time spent swapping and saving paths: 0.02
===
 xx |	 e000	e001	e002	e003	e004
 -- |     -----------------------------------
p74 |	 1.00	----	----	----	----
p75 |	 ----	0.39	0.61	----	----
p73 |	 ----	0.61	0.39	----	----
p69 |	 ----	----	----	1.00	----
p66 |	 ----	----	----	----	----
===
===
shooting wf in ensembles: 003 with paths: 69 and worker: 0
------- infinity 60 END -------

------- infinity 61 START -------
DEBUG this should not happen outside of wirefencing
shooted wf in ensembles: 003 with paths: 69 -> 76 with status: ACC and worker: 0 total time: 23.24
cobra0 ['0', '0']
time spent swapping and saving paths: 0.02
===
 xx |	 e000	e001	e002	e003	e004
 -- |     --------------------------

------- infinity 73 START -------
DEBUG this should not happen outside of wirefencing
shooted wf in ensembles: 004 with paths: 77 -> 92 with status: ACC and worker: 1 total time: 39.33
cobra0 ['1', '1']
time spent swapping and saving paths: 0.03
===
 xx |	 e000	e001	e002	e003	e004
 -- |     -----------------------------------
p89 |	 1.00	----	----	----	----
p90 |	 ----	1.00	----	----	----
p91 |	 ----	----	1.00	----	----
p84 |	 ----	----	----	----	----
p92 |	 ----	----	----	----	1.00
===
===
shooting sh in ensembles: 000 with paths: 89 and worker: 1
------- infinity 73 END -------

------- infinity 74 START -------
DEBUG this should not happen outside of wirefencing
shooted sh in ensembles: 000 with paths: 89 -> 93 with status: ACC and worker: 1 total time: 5.33
cobra0 ['1', '1']
time spent swapping and saving paths: 0.01
===
 xx |	 e000	e001	e002	e003	e004
 -- |     -----------------------------------
p93 |	 1.00	----	----	----	----
p90 |	 ----	1.00	----	----	----
p91 |	 ----	----	1.00

------- infinity 87 START -------
DEBUG this should not happen outside of wirefencing
shooted wf in ensembles: 004 with paths: 104 -> 107 with status: ACC and worker: 1 total time: 70.19
cobra0 ['1', '1']
time spent swapping and saving paths: 0.05
===
 xx |	 e000	e001	e002	e003	e004
 -- |     -----------------------------------
p96 |	 1.00	----	----	----	----
p105 |	 ----	0.42	0.58	----	----
p106 |	 ----	0.58	0.42	----	----
p102 |	 ----	----	----	----	----
p107 |	 ----	----	----	----	1.00
===
===
DEBUG this should not happen outside of wirefencing
shooting sh in ensembles: 000 001 with paths: 96 105 and worker: 1
------- infinity 87 END -------

------- infinity 88 START -------
DEBUG this should not happen outside of wirefencing
shooted wf in ensembles: 003 with paths: 102 -> 108 with status: ACC and worker: 0 total time: 29.75
cobra0 ['0', '0']
time spent swapping and saving paths: 0.02
===
 xx |	 e000	e001	e002	e003	e004
 -- |     -----------------------------------
p96 |	 ----	----

------- infinity 101 START -------
DEBUG this should not happen outside of wirefencing
shooted wf in ensembles: 004 with paths: 119 -> 125 with status: ACC and worker: 0 total time: 34.96
cobra0 ['0', '0']
===
 xx |	 e000	e001	e002	e003	e004
 -- |     -----------------------------------
p123 |	 1.00	----	----	----	----
p124 |	 ----	0.44	0.56	----	----
p116 |	 ----	0.14	0.14	0.72	----
p122 |	 ----	0.29	0.19	0.18	0.34
p125 |	 ----	0.13	0.11	0.11	0.66
===
===
------- infinity 101 END -------



In [11]:
live_trajs = [traj.path_number for traj in state._trajs[:-1]] # state.live_paths()

print('--------------------------------------------------')
print('live trajs:', live_trajs, f'after {stopping} cycles')
print('==================================================')
print('xxx | 000        001     002     003     004     |')
print('--------------------------------------------------')
for key, item in traj_num_dic.items():
    print(f'{key:03.0f}', "|" if key not in live_trajs else '*',
          '\t'.join([f'{item0:02.2f}' if item0 != 0.0 else '---' for item0 in item['weight'][:-1]])
         ,'\t', "|" if key not in live_trajs else '*')

--------------------------------------------------
live trajs: [123, 124, 116, 122, 125] after 100 cycles
xxx | 000        001     002     003     004     |
--------------------------------------------------
000 | ---	5.00	---	---	--- 	 |
001 | ---	---	0.68	0.32	--- 	 |
002 | ---	---	0.32	0.68	--- 	 |
003 | ---	---	---	---	18.00 	 |
004 | 3.00	---	---	---	--- 	 |
005 | ---	---	0.94	5.06	--- 	 |
006 | ---	---	3.06	7.94	--- 	 |
007 | 4.00	---	---	---	--- 	 |
008 | ---	2.00	---	---	--- 	 |
009 | 3.00	---	---	---	--- 	 |
010 | ---	10.00	---	---	--- 	 |
011 | 8.00	---	---	---	--- 	 |
012 | ---	---	2.00	---	--- 	 |
013 | ---	---	1.58	4.42	--- 	 |
014 | ---	0.26	0.55	2.97	13.22 	 |
015 | ---	0.89	1.79	1.08	1.24 	 |
016 | 5.00	---	---	---	--- 	 |
017 | ---	2.88	1.12	---	--- 	 |
018 | ---	0.58	0.58	10.30	2.54 	 |
019 | 3.00	---	---	---	--- 	 |
020 | ---	2.00	---	---	--- 	 |
021 | 3.00	---	---	---	--- 	 |
022 | ---	3.66	2.34	---	--- 	 |
023 | ---	8.16	5.61	4.23	--- 	 |
024 | 6.00	---	---	---	---

Things for the future:
* do internal structure.! [anders lukas]
* worker structure pin -> worker level [daniel] [ok(?)]
* save paths [daniel] [ok]
* run wire fencing [daniel] [ok]
* initialization [titus]
* get gpus [titus, anders]
* restructure run_md function in client.submit(run_md, *args) to not take in "large" traj class objects but rather i.e. trajectory index number? -> speed up process. 
* restart
* tests
* check that we don't do double swap when adding the two paths from the zero swap move

2nd of november